Add your import statements and the database connection statements in the below code block

Database file path = '/course/data/CSE-578/dinofunworld.db'

In [320]:
#your code here
import sqlite3
from datetime import datetime as dt

db_filename = "dinofunworld.db"
connect = sqlite3.connect(db_filename)
cursor = connect.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables_names = cursor.fetchall()
print(tables_names)
for name in tables_names:
    cursor.execute(f"SELECT * FROM {name[0]}")
    table_cols = [description[0] for description in cursor.description]
    print(table_cols)

[('checkin',), ('attraction',), ('sequences',)]
['index', 'visitorID', 'timestamp', 'attraction', 'duration', 'type']
['index', 'AttractionID', 'Name', 'Region', 'Category', 'type']
['index', 'visitorID', 'sequence']


**Question 1:** 

What is the most popular attraction to visit in the park?

**Note:** Your output should be the name of the attraction. Please print your output in the cell below.

In [332]:
### TEST FUNCTION: test_question1
# DO NOT REMOVE THE LINE ABOVE
# your code here
query = """
SELECT Name 
FROM attraction 
WHERE attraction.AttractionID = (
    SELECT attraction
    FROM checkin
    GROUP BY attraction
    HAVING count(attraction) = (
        SELECT Max(x)
        FROM(
            SELECT count(attraction) as x
            FROM checkin
            GROUP BY attraction
        )
    )
)
"""
cursor.execute(query)
name = cursor.fetchone()[0]
print(name)

Atmosfear


**Question 2:** 

What ride (note that not all attractions are rides) has the second longest average visit time?

**Note:** Your output should be the name of the ride. Please print your output in the cell below.


In [324]:
### TEST FUNCTION: test_question2
# DO NOT REMOVE THE LINE ABOVE
# your code here

cursor.execute("SELECT Name FROM attraction WHERE Category LIKE '%ride%'")
rides = cursor.fetchall()

ride_time = {}
for ride in rides:
    ride_time[ride[0]] = 0

ride_count_and_total_time = dict()
for ride in rides:
    ride_count_and_total_time[ride[0]] = (0,0)
query = """
SELECT Name, duration
FROM attraction, checkin
WHERE Category LIKE '%ride%' and attraction.AttractionID = checkin.attraction and checkin.type = 'actual'
"""
cursor.execute(query)
ride_duration = cursor.fetchall()
for category, duration in ride_duration:
    if duration is not None and dt.strptime(duration, "%H:%M:%S").time():
        time = dt.strptime(duration, "%H:%M:%S").time()
        duration_seconds = time.hour * 3600 + time.minute * 60 + time.second
        ride_count = ride_count_and_total_time[category][0] + 1
        ride_total_time = ride_count_and_total_time[category][1] + duration_seconds
        ride_count_and_total_time[category] = (ride_count, ride_total_time)

rides_averages = {}
for key, value in ride_count_and_total_time.items():
    ride_count = value[0]
    ride_total_time = value[1]
    rides_averages[key] = ride_total_time/ride_count

sorted_avg_list = [value for value in rides_averages.values()]
sorted_avg_list.sort()
second_avg_time = sorted_avg_list[-2]

for key, value in rides_averages.items():
    if value == second_avg_time:
        print(key)

Flight of the Swingodon


**Question 3:** 

Which Fast Food offering in the park has the fewest visitors?

**Note:** Your output should be the name of the fast food offering. Please print your output in the cell below.

In [326]:
### TEST FUNCTION: test_question3
# DO NOT REMOVE THE LINE ABOVE
# your code here

# query = """
# SELECT Name 
# FROM attraction 
# WHERE attraction.AttractionID = (
#     SELECT attraction
#     FROM checkin
#     GROUP BY attraction
#     HAVING count(attraction) = (
#         SELECT Min(x)
#         FROM(
#             SELECT count(checkin.attraction) as x
#             FROM checkin, attraction
#             WHERE attraction.Category LIKE '%Food%' and attraction.AttractionID = checkin.attraction
#             GROUP BY checkin.attraction
#         )
#     )
# )
# """

query = """
SELECT Name
FROM attraction, checkin
WHERE attraction.Category LIKE '%Food%' and attraction.AttractionID = checkin.attraction
GROUP BY Name
ORDER BY count(Name) ASC
"""
cursor.execute(query)
seq = cursor.fetchone()[0]
print(seq)

Theresaur Food Stop
